# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
weatherdf = pd.read_csv("../WeatherPy/output_data/cities.csv")
del weatherdf["Unnamed: 0"]
weatherdf.head()

,City,Lat,Lng,Max Temp (F),Humidity,Cloudiness,Windspeed (mph),Country,Date
0,Ushuaia,-54.80,-68.30,33.80,95,40,29.97,AR,1595979630
1,Morgan City,29.70,-91.21,86.00,83,75,6.26,US,1595979631
2,Samarai,-10.62,150.67,78.55,87,46,15.82,PG,1595979631
3,Souillac,-20.52,57.52,69.80,88,40,5.82,MU,1595979631
4,Hobyo,5.35,48.53,77.32,81,3,20.54,SO,1595979631


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [22]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude as locations
locations = weatherdf[["Lat", "Lng"]]
# Store humidity as the weight
humidity = weatherdf["Humidity"]

#create figure
fig=gmaps.figure(center=(0,10), zoom_level = 1.75)

#create heat layer & add
humid_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=3)
fig.add_layer(humid_layer)

#display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [48]:
#Narrow data to ideal weather conditions
    #Max temperature between 70-80 degF, Humidity less than 70%, Wind speed less than 10mph, Cloudiness between 10-60%
idealdf = weatherdf.loc[(weatherdf["Windspeed (mph)"]<10) & (weatherdf["Humidity"]<70) & (weatherdf["Max Temp (F)"]>70) & \
                     (weatherdf["Max Temp (F)"]<80) & (weatherdf["Cloudiness"]<61) & (weatherdf["Cloudiness"]>9)].dropna()    
idealdf   

,City,Lat,Lng,Max Temp (F),Humidity,Cloudiness,Windspeed (mph),Country,Date
121,Sulina,45.16,29.65,73.17,57,14,9.40,RO,1595979650
134,Urucará,-2.54,-57.76,78.91,53,19,0.78,BR,1595979652
159,Yellowknife,62.46,-114.35,73.99,43,20,8.05,CA,1595979557
273,Tynda,55.17,124.72,75.58,69,45,3.02,RU,1595979675
293,Solnechnyy,50.72,136.63,79.75,65,14,3.51,RU,1595979678
309,Kirkland Lake,48.14,-80.04,71.60,68,20,6.93,CA,1595979680
310,Naghadeh,36.96,45.39,70.05,51,23,0.81,IR,1595979680
427,Erenhot,43.65,111.98,75.31,33,28,4.63,CN,1595979698
439,Alta Floresta,-9.88,-56.09,76.30,37,10,3.29,BR,1595979700
553,High Level,58.52,-117.14,71.60,30,20,3.36,CA,1595979717


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [53]:
#create and reorganize df
#add new column for hotel name to hotel df
hotel_df = idealdf
hotel_df.reset_index(drop=True, inplace=True)
hotel_df['Hotel Name']=""
hotel_df.head()

#set parameters
params = {"radius":5000,
         "keyword":"hotel",
         "key":g_key}

#loop to find hotels
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params['location'] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    request = requests.get(base_url, params=params).json()
    
    #try/except loop in case of missing data to fill out hotel names
    try:
        hotel_df.loc[index, "Hotel Name"] = request["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing Data")
        pass
hotel_df

,City,Lat,Lng,Max Temp (F),Humidity,Cloudiness,Windspeed (mph),Country,Date,Hotel Name
0,Sulina,45.16,29.65,73.17,57,14,9.40,RO,1595979650,Hotel Plutitor Carpatia Sf.Constantin
1,Urucará,-2.54,-57.76,78.91,53,19,0.78,BR,1595979652,Hotel Eldorado
2,Yellowknife,62.46,-114.35,73.99,43,20,8.05,CA,1595979557,The Explorer Hotel
3,Tynda,55.17,124.72,75.58,69,45,3.02,RU,1595979675,Gostinitsa Safari
4,Solnechnyy,50.72,136.63,79.75,65,14,3.51,RU,1595979678,"Dom Otdykha ""Pastoral'"""
5,Kirkland Lake,48.14,-80.04,71.60,68,20,6.93,CA,1595979680,Microtel Inn & Suites by Wyndham Kirkland Lake
6,Naghadeh,36.96,45.39,70.05,51,23,0.81,IR,1595979680,MOALLEM HOTEL
7,Erenhot,43.65,111.98,75.31,33,28,4.63,CN,1595979698,Erlian Pacific International Hotel
8,Alta Floresta,-9.88,-56.09,76.30,37,10,3.29,BR,1595979700,Lisboa Palace Hotel
9,High Level,58.52,-117.14,71.60,30,20,3.36,CA,1595979717,Best Western Plus Mirage Hotel & Resort


In [55]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [57]:
# Add marker layer ontop of heat map
fig = gmaps.figure(center=(0, 10), zoom_level=1.75)
markers = gmaps.marker_layer(
    locations,
    info_box_content = hotel_info
)
fig.add_layer(markers)
fig.add_layer(humid_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))